# Tradycyjna analiza tekstu

W ramach pierwszej części ćwiczeń przyjrzymy się tradycyjnym metodom analizy danych tekstowych i zapoznamy się z technikami:

- tokenizacji
- lematyzacji
- rozkładu semantycznego
- reprezentacji TF-IDF

Ćwiczenia wykonamy przy użyciu znakomitej biblioteki [SpaCy](http://spacy.io). Konieczne jest zatem zainstalowanie biblioteki oraz ściągnięcie modelu języka. W ćwiczeniach wykorzystamy zarówno mode języka angielskiego, jak i języka polskiego. Przed przystąpieniem do realizacji ćwiczenia niezbędne jest wykonanie następujących kroków:

```bash
pip install -U spacy

python -m spacy download en_core_web_md

python -m spacy download pl_core_news_md

python -m spacy validate
```

## Tokenizacja i lematyzacja

W pierwszym kroku załadujemy model języka i przetworzymy tekst dokonując podziału na tokeny, a następnie sprawdzimy, jakie informacje możemy wyciągnąć z każdego tokenu.

In [ ]:
import spacy

nlp = spacy.load('pl_core_news_md')

# lista wszystkich słów które posiadają zdefiniowane wektory
allWords = [
    orth
    for orth in nlp.vocab.vectors 
    if nlp.vocab[orth].has_vector
]

print(f"Liczba słów w słowniku: {len(allWords)}")
print(f"Słownik: {[nlp.vocab[orth].text for orth in allWords[:10]]}")

In [ ]:
doc = 'Król Karol Ubogowłosy XII kupił królowej Karolinie pięć korali koloru koralowego na urodziny 1 grudnia!'

for t in nlp(doc):
    print(f"{t.text:<12}: {t.lemma_:<10}: {t.pos_:<7}: {t.is_digit}: {t.is_punct}: {t.is_oov}")

### zadanie samodzielne

Zapoznaj się z [dokumentacją klasy Token](https://spacy.io/api/token) i dla poniższego zdania wyświetl, dla każdego tokenu, następujące informacje:

- tekst tokenu
- symbol części mowy (zarówno ogólny jak i szczegółowy)
- znormalizowaną formę tokenu
- formę słownikową tokenu
- informację czy token jest cały napisany kapitalikami

In [ ]:
doc = 'Rola Roberta DeNiro w tym filmie była NIESAMOWITA!!!'

for t in nlp(doc):
    print(f"")

In [ ]:
doc = 'na stronie www.amazon.com kupiłem 2 kindle (za 90,34 EUR) od morzy@gmail.com'

for t in nlp(doc):
    print(f"{t.text:<17}: {t.like_url}: {t.like_email}: {t.like_num}: {t.is_left_punct}: {t.is_right_punct}")

### zadanie samodzielne

Przygotuj przykład zdania, które będzie zawierało następujące elementy:

- fragment przypominający e-mail
- fragment przypominający kwotę pieniężną
- fragment przypominający numer NIP
- słowo o zdecydowanie pozytywnym wydźwięku
- słowo o zdecydowanie negatywnym wydźwięku

a następnie wyświetl dla każdgo tokenu następujące informacje:

- czy dany token przypomina e-mail?
- czy dany token przypomina liczbę?
- wydźwięk (sentyment) tokenu

In [ ]:
doc = ''

for t in nlp(doc):
    print(f"")

## Ekstrakcja nazwanych encji

NER (ang. named entity recognition) to zadanie NLP polegające na zidentyfikowaniu w tekście tokenów reprezentujących wystąpienia określonych kategorii obiektów świata rzeczywistego, np. osób, miejsc, organizacji, dat.

In [ ]:
doc = """W poznańskiej Katedrze znajduje się 
symboliczny grobowiec pierwszych władców Polski:
Mieszka I i Bolesława I Chrobrego. 
"""

from spacy import displacy
displacy.render(nlp(doc), style="ent")

### zadanie samodzielne

Znajdź [schemat etykiet użyty w modelu języka](https://spacy.io/models/pl) (*hint*: szukaj Labeling Scheme) i napisz akapit, który będzie zawierał wystąpienie każdej encji ze schematu etykiet.

In [ ]:
doc = """
"""

from spacy import displacy
displacy.render(nlp(doc), style="ent")

## Rozkład syntaktyczny zdania

Zadanie rozkładu syntaktycznego polega na zidentyfikowaniu ról, jaką poszczególne tokeny pełnią w zdaniu (identyfikacja części zdania), oraz znalezieniu zależności między tymi częściami.

In [ ]:
doc = "Czarna krowa w kropki bordo gryzła trawę kręcąc mordą."

displacy.render(nlp(doc), style="dep")

In [ ]:
for t in nlp(doc):
    print(f"{t.text:<10} {t.dep_:<8} {t.head.text:<10} {t.head.pos_}   children: {[child for child in t.children]}")

### zadanie samodzielne

Zapoznaj się z definicją poszczególnych [rodzajów zależności](https://universaldependencies.org/pl/index.html) a następnie postaraj się zidentyfikować w poniższym zdaniu pary słów reprezentujące podmiot i orzeczenie.

In [ ]:
doc = "Jan powiedział, że chce obejrzeć film, ale Anna, niechętna temu pomysłowi, zaproponowała wyjście do kawiarni"

for t in nlp(doc):
    if t.dep_ == 'xxx uzupełnij xxx':
        print(f"{t.text:<15} {t.dep_:<8}  {t.head.text:<15}")

## Wektory TF-IDF i częstotliwość słów

Funkcja `zipf_frequency` pozwala na wyznaczenie, w pewnym przybliżeniu, częstości występowania danego słowa w języku. Wartość zwracana przez funkcję to logarytm częstości, słowa rzadkie mają wartość w przedziale [0-1], podczas gdy najpopularniejsze słowa (się, w, na, nie) mają wartośc w przedziale [7-8].

In [ ]:
import numpy as np
from wordfreq import zipf_frequency

for random_word in [nlp.vocab[allWords[i]].text for i in np.random.randint(len(allWords), size=10)]:
    print(f"{random_word}, {zipf_frequency(random_word,'pl')}")

Poniższy kod wykorzystuje bibliotekę `BeautifulSoup` do pobrania całego opisu zamieszczonego na stronie [pl.wikipedia.org/wiki/Poznań](https://pl.wikipedia.org/wiki/Pozna%C5%84) Następnie tekst jest łączony do postaci jednego łańcucha znaków i usuwane są wszystkie odnośniki do literatury w postaci [1], [2], itd.

In [ ]:
from bs4 import BeautifulSoup
import requests
import re

respond = requests.get("https://pl.wikipedia.org/wiki/Poznań")
soup = BeautifulSoup(respond.text, "html")
page = soup.find_all('p')

raw_text = ''.join([re.sub(r'\[\d+\]', '', paragraph.text) for paragraph in page])

print(raw_text)

W dalszej części ćwiczenia przyjmiemy, że jednostką analizy (dokumentem) jest jedno zdanie. Przyjrzymy się dwóm tradycyjnym sposobom reprezentacji zdań w przestrzeni wektorowej:

- modelowi bag-of-words
- modelowi tf-idf

Do wyznaczenia wektorów wykorzystamy gotowe klasy z biblioteki `scikit-learn`. Wykorzystamy także gotową listę słów typu *stopword* dla języka polskiego dostępną pod [tym adresem](https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt)

In [ ]:
import urllib.request

url = "https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt"
stop_words = urllib.request.urlopen(url).read().decode('utf-8').split()

print(stop_words)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

phrases = raw_text.split('.')

In [ ]:
count_vectorizer = CountVectorizer(min_df=0.01, max_df=0.1, stop_words=stop_words)

count_vectorized_phrases = count_vectorizer.fit_transform(phrases)
print(f"Phrase: {phrases[3]}")
print(f"Vector representation: {count_vectorized_phrases.todense()[3]}")
print(f"Features: {count_vectorizer.get_feature_names()}")

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.1, stop_words=stop_words)

tfidf_vectorized_phrases = tfidf_vectorizer.fit_transform(phrases)
print(f"Phrase: {phrases[3]}")
print(f"Vector representation: {tfidf_vectorized_phrases.todense()[3]}")
print(f"Features: {tfidf_vectorizer.get_feature_names()}")

### zadanie samodzielne

Wykorzystując zbudowany powyżej model językowy spróbuj wyznaczyć reprezentację wektorową poniższego zdania:

In [ ]:
doc = 'Na terenie Poznania od X wieku nad Wartą znajdowało się centrum wielkopolski'

vectorized_doc = # uzupełnij

print(vectorized_doc.todense())